# map visualization


In [1]:
! pip install folium
! pip install spacy
! pip install geopy

In [2]:
! python -m spacy download en_core_web_sm

import spacy
from geopy.geocoders import Nominatim
import folium

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

     ---------------------------------------- 12.8/12.8 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## function for parsing location data via api

In [3]:
def extract_locations(text):
    if text:
        doc = nlp(text)
        return [ent.text for ent in doc.ents if ent.label_ == "GPE"]  # GPE (Geopolitical Entity)
    return []

def get_coordinates(location):
    geolocator = Nominatim(user_agent="geoapi")
    try:
        loc = geolocator.geocode(location)
        if loc:
            return (loc.latitude, loc.longitude)
    except Exception as e:
        print(f"Error getting coordinates for {location}: {e}")
    return None


# retriving & mapping location values 

In [4]:
import pandas as pd
news_data = pd.read_csv('work\\fox_news.csv')
# Extract locations from the Corpus
news_data['locations'] = news_data['Corpus'].apply(extract_locations)

# Get coordinates for each location
news_data['coordinates'] = news_data['locations'].apply(lambda locs: [get_coordinates(loc) for loc in locs if get_coordinates(loc)])

# Drop rows where no valid coordinates were found
news_data = news_data[news_data['coordinates'].map(len) > 0]




Error getting coordinates for fbi york: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=fbi+york&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002EEEDABF400>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))
Error getting coordinates for los angeles: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=los+angeles&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002EEEDB1AC40>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))
Error getting coordinates for philadelphia: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=philadelphia&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002EEEDB1A9A0>, 'Connec

In [5]:
# Create a folium map
map = folium.Map(location=[0, 0], zoom_start=2)

# Add markers to the map
for _, row in news_data.iterrows():
    for coord in row['coordinates']:
        if coord:
            folium.Marker(location=coord, popup=row['Title']).add_to(map)

# Save map to HTML
map.save("work\\news_geospatial_analysis.html")